# Preprocessing Jupiter Notebook: 
gets the data scraped by the scrape.py scripts, creates dataframes from the lists and preprocesses for the machine learning model
1. part 1: create the dataframe from the scraped data

In [1]:
import pandas as pd
from scrape import customer_service_reviews, comments, dates, satisfaction_reviews, speed_reviews, reliability_reviews, customer_service_reviews

In [2]:
#combine all list values to a df:
#create the dictionary that will make up the dataframe
df_dict={"Dates": dates, "Comments": comments, "Customer Service": customer_service_reviews, "Satisfaction Reviews": satisfaction_reviews, "Speed Reviews": speed_reviews, "Reliability Reviews": reliability_reviews}

#as certain columns are shorter than others use a trick to create a df with unequal arrays:https://stackoverflow.com/questions/19736080/creating-dataframe-from-a-dictionary-where-entries-have-different-lengths
df = pd.DataFrame.from_dict(df_dict, orient='index')
df=df.transpose()

In [3]:
#convert the dates columns from string to datetime object:
df["Dates"]= pd.to_datetime(df["Dates"])

In [4]:
#save date to csv:
df.to_csv('raw_data.csv')

2. data cleaning: Dealing with the null values 

In [15]:
#first read csv dropped unnamed columns and inspect the first few rows
raw_df= pd.read_csv("raw_data.csv")
raw_df.drop("Unnamed: 0", axis=1, inplace=True)
print(raw_df.shape) 

(1960, 6)


In [16]:
raw_df.head()

,Dates,Comments,Customer Service,Satisfaction Reviews,Speed Reviews,Reliability Reviews
0,2022-11-15,Moved back to the UK end of August and got Vir...,1.0,1.0,1.0,1.0
1,2022-11-14,"A truly attrocious service, both in terms of b...",1.0,1.0,1.0,1.0
2,2022-11-13,They make it as hard as they can for you to ca...,1.0,1.0,2.0,2.0
3,2022-11-11,Pay for the 350Mbps package but only ever mana...,1.0,1.0,3.0,2.0
4,2022-11-11,The worst customer service:\r-The bots ask irr...,1.0,1.0,3.0,2.0


check if any of the comment rows are empty. If they are we are forced to drop this row as the main project revolves around nlp

In [18]:
raw_df['Comments'].isnull().values.any()

True

drop any reviews that have no text as of now use to us.

In [21]:
raw_df.dropna(subset=['Comments'], inplace=True)

In [22]:
raw_df.shape

(1959, 6)

Now that rows with missing comments have been dropped lets look at the rest of the missing data. We will deal with the missing values bu using scikit learn's simple imputer, that will impute the missing data in each column for us accoprding the to the most common value

In [24]:
raw_df['Customer Service'].isnull().values.any()

True

In [26]:
from sklearn.impute import SimpleImputer
import numpy as np

imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer = imputer.fit(raw_df[['Customer Service']])


True